In [7]:
#Import Libraries

import geopandas as gpd
import pandas as pd
from math import cos, sin, radians
from shapely.geometry import Point, LineString, Polygon
from multiprocessing import Pool
import numpy as np

In [ ]:
#Select study extent shapefile to create the fishnet from
polygon_layer = gpd.read_file('C:/Users/cowboy/Documents/WidthMetric/Sample_Data/StudyExtent.shp')

In [ ]:
#Project polygon to meters
polygon_layer = polygon_layer.to_crs(epsg=32612)

In [ ]:
### Create your fishnet ###

# Read the shapefile using geopandas
shapefile = polygon_layer

# Define the desired cell size for the fishnet grid
cell_width = 30  # Replace with your desired cell width in the map units
cell_height = 30  # Replace with your desired cell height in the map units

# Calculate the extent of the fishnet grid based on the shapefile's bounding box
xmin, ymin, xmax, ymax = shapefile.total_bounds

# Calculate the actual number of rows and columns based on the desired cell size
num_rows = int((ymax - ymin) / cell_height)
num_cols = int((xmax - xmin) / cell_width)

# Create the fishnet grid polygons
fishnet_polys = []
centroids = []
for row in range(num_rows):
    y_start = ymin + (row * cell_height)
    y_end = y_start + cell_height
    for col in range(num_cols):
        x_start = xmin + (col * cell_width)
        x_end = x_start + cell_width
        poly = Polygon([(x_start, y_start), (x_end, y_start), (x_end, y_end), (x_start, y_end)])
        fishnet_polys.append(poly)
        centroid = poly.centroid
        centroids.append(centroid)

# Create a geodataframe from the fishnet polygons
fishnet_grid = gpd.GeoDataFrame(geometry=fishnet_polys, crs=shapefile.crs)

# Create a geodataframe for the centroids
centroids_gdf = gpd.GeoDataFrame(geometry=centroids, crs=shapefile.crs)

# Save the fishnet grid and centroids as separate shapefiles
output_grid_path = 'C:/Users/cowboy/Documents/WidthMetric/Sample_Data/FishnetGrid30m.shp'
output_centroids_path = 'C:/Users/cowboy/Documents/WidthMetric/Sample_Data/FishnetCentroids30m.shp'
fishnet_grid.to_file(output_grid_path)
centroids_gdf.to_file(output_centroids_path)